> #  **Topic modeling fine tuning**

---

 

In [ ]:
#from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
#from sklearn.decomposition import NMF, LatentDirichletAllocation
#from sklearn.model_selection import ShuffleSplit
#from sklearn.feature_selection import chi2
#from sklearn.decomposition import PCA
#
#import numpy as np
#import pandas as pd
#import random
#import nltk
import matplotlib.pyplot as plt 
    
x = [1,2,3] 
y = [2,4,1] 
plt.plot(x, y) 
plt.xlabel('x - axis') 
plt.ylabel('y - axis') 
plt.title('My first graph!') 
plt.show() 

---
**D**entro del minado de datos existe la tarea de identificar un conjunto de palabras que representen al texto de un `documento` y en esta libreta se mostrara la solucion llamada *Topic Modeling*, el cual utiliza un enfoque probabilistico para resolver esta problematica.

El curso de la libreta es el siguiente:
 - Vectorizacion
 - *Ingenieria de caracteristicas*
 - Aplicar Topic modeling
 - Validacion "*empirica*"
 


Los boletines expedidos por la [pagina](https://presidente.gob.mx/) del presidente mexicano Andres Manuel Lopez Obrador forman el `corpus` usado a lo largo del documento, el cual fue procesado y limpiado en otra libreta. 

El EDA se realizara en la plataforma [Tableau](https://www.tableau.com/es-mx), para esto se guardaran periodicamente algunos valores en un archivo con extencion '.csv'  

In [1]:
corpus = pd.read_json('../primera_iteracion/juve_datos.json',orient='record')


corpus_validacion = corpus.sample(frac=8 / len(corpus) , random_state = 564)
corpus_entrenamiento = corpus.drop(corpus_validacion.index)
corpus_entrenamiento = corpus_entrenamiento.reset_index()

print('Conjunto entrenamiento:')
print(corpus_validacion.head())

print('\nConjunto validacion:')
print(corpus_entrenamiento.head())


#############################################################################
# Nota, este codigo esta por mientras el tokenizado. Borrar esto despues...
stopword_es = nltk.corpus.stopwords.words('spanish')
def remove_stopwords(text):
    texto = text.split()
    text = [word for word in texto if word not in stopword_es]
    text_r = ' '.join(text)
    return text_r
corpus_entrenamiento['Titulo_ns'] = corpus_entrenamiento['Titulo'].apply(lambda x: remove_stopwords(x))
#----------------------------------------------------------------------------

Conjunto entrenamiento:
                                                 Titulo       Fecha
2255  Poderes Ejecutivo y Judicial coordinan esfuerz...  13.02.2020
2692  Presidente analiza terminar construcción de 54...  27.07.2019
1929  Fotogalería – Presidente inaugura Unidad de Me...  31.08.2020
2291  Avanza adhesión de estados al Instituto de Sal...  28.01.2020
1184  Reforma eléctrica fortalecerá a CFE y proteger...  23.10.2021

Conjunto validacion:
   index                                             Titulo       Fecha
0      0  Presidente se reúne en Palacio Nacional con go...  17.04.2023
1      1  Obras complementarias en tramo 1 del Tren Maya...  17.04.2023
2      2  Fonatur y Semarnat rinden informe de avances e...  17.04.2023
3      3  En ruta del Tren Maya, estudiantes de todos lo...  17.04.2023
4      4  Caminos pavimentados comunicarán todas las cab...  17.04.2023


**H**ay que recalcar que los documentos no estan etiquetados, por lo que para el *topic modeling* se esta restringido a **algoritmos no supervisados**. Al algoritmo que se utiliza para conseguir la clasificacion de los topicos se le llamara **modelo**. El corpus de validacion es para realizar una evaluacion empirica de nuestro resultado. 

## Vectorizacion
---

**E**l formato de texto suele ser poco amigable para los algoritmos por lo que es conveniente transformar el texto a un formato numerico que nos permita representar las **caracteristicas numericas** del texto. A esto se le llama vectorizado.

La eleccion de las caracteristicas numericas depende en gran medida del modelo que se este usando. Los vectorizadores mas comunes en PLN son los basados en **frecuencias** y los basados en **algoritmos de aprendizaje**, los primeros nos permiten representar un documento por la frecuencia de sus palabras y se suele decir que los segundos logran capturar el lenguaje semantico de las palabras.    

Intuitivamente se puede pensar que las palabras mas recurrentes representan a un documento, pero esto puede no ser verdad en los casos extremos donde una palabra aparece mucho en el corpus. Para resolver este problema existe el vectorizado *term frequency - inverse document frequency* `(tfidf)` .

In [ ]:
frecuencias_del_corpus = TfidfVectorizer(
    ngram_range = (1,2),
    max_df=0.99999999999, 
    min_df=0
    )

corpus_vectorizado = frecuencias_del_corpus.fit_transform(corpus_entrenamiento.Titulo_ns) 

### Guardando frecuencias 
---
**C**omo en la mayoria de modelos, se busca que nuestro *conjunto de entrenamiento* pueda capturar las caracteristicas **mas significativas** y que el tamaño del *vector caracteristicas* sea **lo mas pequeño posible**

In [2]:
print(frecuencias_del_corpus.stop_words_)

matriz_cruda = corpus_vectorizado.toarray()
tokenizador = frecuencias_del_corpus.build_analyzer()
diccionario_vocabulario = frecuencias_del_corpus.vocabulary_

archivo_documentos ='documentos.csv'
archivo_diccionario='diccionario_token.csv'
archivo_tokens_por_documento = 'palabras_por_documento.csv'
archivo_matriz_cruda = 'matriz_cruda_frecuencias.csv'

##################################################################################
#Escribiendo el archivo 'informacion_documento' 
with open(archivo_documentos , 'w', encoding='utf-8') as archivo:
    archivo.write(f'id_documento,titulo,fecha\n')
    for elemento in corpus_entrenamiento.iloc:
        id_documento =elemento.name
        titulo = elemento.Titulo
        fecha = elemento.Fecha
        archivo.write(f'{id_documento},{titulo},{fecha}\n')

        
#_________________________________________________________________________________


##################################################################################'
# Guardando diccionario de los tokens
with open(archivo_diccionario, 'w', encoding='utf-8') as archivo:
    archivo.write(f'id_token,token\n')
    for (llave) in diccionario_vocabulario:
        archivo.write(f'{llave},{diccionario_vocabulario[llave]}\n')
#_________________________________________________________________________________ 


##################################################################################
#Escribiendo el archivo 'palabras_por_documento' 
with open(archivo_tokens_por_documento, 'w', encoding='utf-8') as archivo:
    archivo.write(f'id_documento,token,frecuencia\n')
    for elemento in corpus_entrenamiento.iloc:
        id_documento =elemento.name
        for token in (tokenizador(elemento.Titulo_ns)):
            id_token =diccionario_vocabulario[token] 
            frecuencia = matriz_cruda[id_documento][id_token]
            archivo.write(f'{id_documento},{token},{frecuencia}\n')
#_________________________________________________________________________________



##################################################################################
#Escribiendo el archivo 'matriz_cruda_frecuencias'
dataframe_corpus = pd.DataFrame(corpus_vectorizado.toarray())
dataframe_corpus.to_csv(archivo_matriz_cruda,index=False)
#Ejemplo_leer = pd.read_csv('frecuencias_del_corpus.csv')
#_________________________________________________________________________________ 

    
print('listo')

set()
listo


## Reduciendo dimencionalidad
---

**C**uando el vector de caracteristicas es muy grande se suele sufrir de *la maldición de la dimensionalidad* (o efecto Hughes), el cual es un problema conocido de la estadistica y probabilidadEn. Para mitigar estos efectos la solucion inmediata es reducir el numero de caracteristicas en nuestro vector. Los metodos mas populares son el **PCA** y usando una distribucion **chi2**.

In [ ]:
#If 0 < n_components < 1 and svd_solver == 'full', 
#select the number of components such that the amount of variance that 
#needs to be explained is greater than the percentage specified by n_components.
#svd_solver{‘auto’, ‘full’, ‘arpack’, ‘randomized’}

pca_corpus_entrenamiento = PCA(
    n_components= 0.70,
    svd_solver= 'full',
    whiten = False,
    random_state=576
    ) 
pca_corpus_entrenamiento.fit(corpus_vectorizado.toarray())
corpus_frecuencias_reducidas = corpus_frecuencias_reducidas.transform(corpus_vectorizado)

print('listo')

In [ ]:
print('f')

In [ ]:
print(corpus_frecuencias_reducidas)

### Guardando nuevas caracteristicas
---

## Aplicando Topic modeling

**L**as tecnicas de analizar grupos en aprendizaje automatico se conocen como **clustering** y el algoritmo mas popular de clustering es **K-means** el cual usa un enfoque basado en la *distancia* entre vectores de caracteristicas. 

Sin embargo para agrupar documentos existe un metodo muy popular que consiste en calcular dos *distribuciones de probabilidades*, Una de *topicos sobre tokens* y otra de *documentos sobre topicos*. Al algoritmo que calcula estas dos distribuciones se le llama **Latent Dirichlet Allocation**.

In [3]:
lda = LatentDirichletAllocation(
    n_components=n_components,
    max_iter=500,
    learning_method="online",
    learning_offset=50.0,
    random_state=876,
)
lda.fit()

feature = frecuencias_del_corpus.get_feature_names()
for ind, topic in enumerate(lda.components_):
    print('Top 50 words in topic {}'.format(ind))
    print('-'*25)
    topic_ordenado = topic.argsort()[::-1]
    top_50 = topic_ordenado[:50]
    print([feature[i] for i in top_50] , '\n\n')

NameError: name 'n_components' is not defined

### Guardando entropia del topic modeling

## Evaluacion empirica
---